# Convert `GeoTIFFs` in Google Cloud Storage to `Zarr`

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
import zarr
import rioxarray
import gcsfs
import matplotlib.pyplot as plt
import os
import json

In [2]:
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'

## From `GeoTIFFs` to `Zarr`

We use the [xarray](http://xarray.pydata.org/en/stable/io.html#reading-and-writing-files) library to convert `GeoTIFFs` into `Zarr`. 

GeoTIFFs can be opened using [rasterio](http://xarray.pydata.org/en/stable/io.html#rasterio) with this xarray method: `xarray.open_rasterio`. Additionally, you can use [rioxarray](https://corteva.github.io/rioxarray/stable/) for reading GeoTiffs.

To save `xarray.Datasets` as a `Zarr` we can us the [Xarray’s Zarr backend](http://xarray.pydata.org/en/stable/io.html#zarr). [Zarr](http://zarr.readthedocs.io/) is a Python package providing an implementation of chunked, compressed, N-dimensional arrays. Zarr has the ability to read and write xarray datasets directly from / to cloud storage buckets such as Amazon S3 and Google Cloud Storage.

Xarray needs to read all of the zarr metadata when it opens a dataset. With version 2.3, Zarr will support a feature called consolidated metadata, which allows all metadata for the entire dataset to be stored with a single key (by default called `.zmetadata`). This can drastically speed up opening the store. To write consolidated metadata, pass the `consolidated=True` option to the `Dataset.to_zarr` method.

### Argentina SOC data

#### SOC stocks dataset

**Data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/

**Data description:**

The name structure of the files is `Feb19_cstocks_YEAR_030_ll.tif`:
- YEAR: 1982-2017
- The stocks were calculated in the 0 to 30 cm interval. 

**Output data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/soil-tnc-data.zarr/

In [3]:
%%time
local_path = '../data/test.zarr'
project_name = 'soc-platform'
bucket_name = 'vizz-data-transfer'
root = bucket_name+'/SOC_maps/test.zarr'
group = 'experimental-dataset'
private_key = json.loads(os.getenv("PRIVATE_KEY"))
ds_name = 'stocks'
base_url = 'https://storage.googleapis.com/vizz-data-transfer/SOC_maps/SOC_stock/'
gc = gcsfs.GCSFileSystem(project=project_name, token=private_key)

times = pd.date_range("1982", "2018", freq='A-DEC', name="time")
depth = ['0-30']
years = np.arange(1982, 1985, 1).astype(np.str)

for i, year in enumerate(years):
    print(f'Year: {year}')
    url = base_url + 'Feb19_cstocks_' + year + '_030_ll.tif'
    xda = xr.open_rasterio(url).squeeze().drop("band")
    
    # replace all values equal to -9999 with np.nan
    xda = xda.where(xda != -9999.) 
    
    # add time and depth coordinates
    xda = xda.assign_coords({"time": times[i]}).expand_dims('time')#.expand_dims(['time', 'depht'])
    
    # convert to Dataset
    xds = xr.Dataset({ds_name: xda}, attrs=xda.attrs)
    
    # select sub-area
    xds = xds.isel(x=slice(2000, 2100), y=slice(4000, 4100))
    
    # save zarr into Google Cloud Storage bucket
    if i == 0:
        # Save in GCS
        #store = gc.get_mapper(root, check=False, create=True)
        store = gc.get_mapper(root)
        xds.to_zarr(store=store, group=group, mode='w', consolidated=True)
        # consolidate metadata at root
        zarr.consolidate_metadata(store)
        #c = gc.exists(f"{root}/.zmetadata")
        #print(f"{root} is consoldiated? {c}")
        with zarr.open(store, mode='r') as z:
            print(z.tree())
        
        # Save locally
        xds.to_zarr(local_path, group=group, mode='w', consolidated=True)
        # consolidate metadata at root
        #zarr.consolidate_metadata(local_path)
        #with zarr.open(local_path, mode='r') as z:
        #    print(z.tree())
    else:
        # Save in GCS
        #store = gc.get_mapper(root, check=True, create=False)
        xds.to_zarr(store=store, group=group, mode='a', append_dim='time', consolidated=True)
        # consolidate metadata at root
        zarr.consolidate_metadata(store)
        #c = gc.exists(f"{root}/.zmetadata")
        #print(f"{root} is consoldiated? {c}")
        with zarr.open(store, mode='r') as z:
            print(z.tree())
        
        # Save locally
        xds.to_zarr(local_path, group=group, append_dim='time', consolidated=True)
        # consolidate metadata at root
        #zarr.consolidate_metadata(local_path)
        #with zarr.open(local_path, mode='r') as z:
        #    print(z.tree())

Year: 1982
/
 ├── stocks (1, 100, 100) float32
 ├── time (1,) int64
 ├── x (100,) float64
 └── y (100,) float64
Year: 1983
/
 ├── stocks (1, 100, 100) float32
 ├── time (1,) int64
 ├── x (100,) float64
 └── y (100,) float64
Year: 1984
/
 ├── stocks (1, 100, 100) float32
 ├── time (1,) int64
 ├── x (100,) float64
 └── y (100,) float64
CPU times: user 13.4 s, sys: 4.58 s, total: 18 s
Wall time: 2min 52s


In [ ]:
local_path = '../data/test.zarr'
group = 'experimental-dataset'
ds_zarr = xr.open_zarr(local_path, group=group)
ds_zarr

In [4]:
# Connect to GS
project_name = 'soc-platform'
bucket_name = 'vizz-data-transfer'
root = bucket_name+'/SOC_maps/test.zarr'
group = 'experimental-dataset'
private_key = json.loads(os.getenv("PRIVATE_KEY"))

gc = gcsfs.GCSFileSystem(project=project_name, token=private_key)
store = gc.get_mapper(root)
# Check zarr is consolidated
#consolidated = gc.exists(f'{root}/.zmetadata')
# Cache the zarr store
#cache = zarr.LRUStoreCache(store, max_size=None)
# Return cached zarr group
ds_gcs = xr.open_zarr(store=store, group=group, consolidated=True)
ds_gcs

<xarray.Dataset>
Dimensions:  (time: 1, x: 100, y: 100)
Coordinates:
  * time     (time) datetime64[ns] 1982-12-31
  * x        (x) float64 -69.91 -69.91 -69.91 -69.9 ... -69.68 -69.68 -69.67
  * y        (y) float64 -31.37 -31.37 -31.38 -31.38 ... -31.6 -31.61 -31.61
Data variables:
    stocks   (time, y, x) float32 dask.array<chunksize=(1, 100, 100), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:     Area
    TIFFTAG_SOFTWARE:  GRASS GIS 7.6.0 with GDAL 2.2.2
    crs:               +init=epsg:4326
    is_tiled:          1
    nodatavals:        [-9999.0]
    offsets:           [0.0]
    res:               [0.0023966782079451365, 0.0023966782079451365]
    scales:            [1.0]
    transform:         [0.0023966782079451365, 0.0, -74.70544001621114, 0.0, ...

[to_zarr append with gcsmap does not work properly #3251](https://github.com/pydata/xarray/issues/3251)